# Projeto de Porfólio

## Aplicando técnicas de Feature Engineering em um dataset não tratado

In [2]:
# Imports

import numpy as np
import pandas as pd

Primeiro, precisamos de nosso dataset. Iremos utilizar [esse dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), que é um conjunto de dados de filmes e séries da Netflix.

Então, vamos carregá-lo:

In [3]:
df = pd.read_csv("netflix_titles.csv")

Vamos usar head e describe para observar as features:

In [16]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [17]:
df.describe()

,release_year
count,8807.000000
mean,2014.180198
std,8.819312
min,1925.000000
25%,2013.000000
50%,2017.000000
75%,2019.000000
max,2021.000000


Nosso único feature numérico é o ano de lançamento. Não há muito o que dizer, o mínimo é 1925 e o máximo, 2021, o que é esperado. Podemos usar include para verificar as features categóricas:

In [18]:
df.describe(include=['object'])

,show_id,type,title,director,cast,country,date_added,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8803,8804,8807,8807
unique,8807,2,8807,4528,7692,748,1767,17,220,514,8775
top,s1,Movie,Dick Johnson Is Dead,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",TV-MA,1 Season,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,3207,1793,362,4


Agora, podemos verificar quantos valores nulos nós temos:

In [19]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

Há muitos valores nulos em director, cast, country e date_added. É uma boa observarmos!

### País (country)

Como se trata de uma feature categórica, vamos observar seus valores únicos:

In [20]:
df['country'].unique()

array(['United States', 'South Africa', nan, 'India',
       'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia',
       'United Kingdom', 'Germany, Czech Republic', 'Mexico', 'Turkey',
       'Australia', 'United States, India, France', 'Finland',
       'China, Canada, United States',
       'South Africa, United States, Japan', 'Nigeria', 'Japan',
       'Spain, United States', 'France', 'Belgium',
       'United Kingdom, United States', 'United States, United Kingdom',
       'France, United States', 'South Korea', 'Spain',
       'United States, Singapore', 'United Kingdom, Australia, France',
       'United Kingdom, Australia, France, United States',
       'United States, Canada', 'Germany, United States',
       'South Africa, United States', 'United States, Mexico',
       'United States, Italy, France, Japan',
       'United States, Italy, Romania, United Kingdom',
       'Australia, United States', 'Argentina, Venezuela',
       'United States, United Kin

Podemos contar o número de valores únicos, também:

In [10]:
np.size(df['country'].unique())

749

Como fica evidente, temos 749 valores únicos para países, o que excede bastante o número de países do mundo; o número varia dependendo de quais territórios são considerados países, mas está em torno de 200.

Isso ocorre porque temos algumas listagens possuem mais de um país, pois trata-se de uma série ou filme que envolve múltiplas nacionalidades.

Quando isso ocorre, os países são listados em ordem e separados por vírgula. Claramente não é ordem alfabética - como é possível verificar por "United States" vir antes de "Colombia", por exemplo. Então assumirei que é uma ordem de relevância, e removerei todos os países seguintes, mantendo apenas os mais relevantes.

Quanto aos nulos, é impossível verificar um-por-um, e não há um jeito simples de preenchê-los. Então, por simplicidade, só trocarei por "Unknown" ("Desconhecido"), para que eles sigam o formato do restante dos países:

In [11]:
df['country'].fillna('Unknown',inplace=True)

Agora, vamos remover os países separados por vírgula:

In [23]:
df['country'].replace('.*(?<=,)',"",regex=True,inplace=True)

In [24]:
df['country'].replace(',',"",regex=True,inplace=True)

Percebi que também há alguns países que são só uma string vazia. Portanto, vamos trocá-los por "Unknown":

In [26]:
df['country'].replace('',"Unknown",regex=True,inplace=True)

In [27]:
df['country'].unique()

array(['United States', 'South Africa', 'Unknown', 'India', ' Ethiopia',
       'United Kingdom', ' Czech Republic', 'Mexico', 'Turkey',
       'Australia', ' France', 'Finland', ' United States', ' Japan',
       'Nigeria', 'Japan', 'France', 'Belgium', ' United Kingdom',
       'South Korea', 'Spain', ' Singapore', ' Canada', ' Mexico',
       ' Venezuela', ' Hong Kong', 'Russia', 'Canada', 'Hong Kong',
       ' Germany', ' South Korea', 'Ireland', ' Nepal', 'Italy',
       ' Greece', 'Argentina', 'Jordan', 'Colombia', ' Australia',
       'Brazil', ' Spain', 'Taiwan', ' Nigeria', ' China', ' Algeria',
       'Poland', 'Germany', 'New Zealand', 'Saudi Arabia', 'Thailand',
       'Indonesia', ' Switzerland', ' Belgium', 'Egypt', 'Malaysia',
       'Israel', ' New Zealand', ' Netherlands', 'Vietnam', ' Italy',
       ' Hungary', 'Sweden', 'China', 'Lebanon', 'Romania', ' Syria',
       'Philippines', 'Iceland', 'Denmark', ' India', ' Indonesia',
       ' United Arab Emirates', ' Denmar

### Diretor (director)

Vamos fazer o mesmo processo:

In [31]:
df['director'].unique()

array(['Kirsten Johnson', nan, 'Julien Leclercq', ..., 'Majid Al Ansari',
       'Peter Hewitt', 'Mozez Singh'], dtype=object)

A lista é muito grande, então vamos ter de analisar de  outra forma. Antes de tudo, vamos trocar os nulos por "Unknown" novamente:

In [38]:
df['director'].fillna('Unknown',inplace=True)

A questão é que podemos ter o mesmo problema que tivemos com country. Para verificar isso, podemos verificar a presença de vírgulas:

In [43]:
a = df['director'].str.findall(r',')

In [46]:
np.unique(a)

array([list([]), list([',']), list([',', ',']), list([',', ',', ',']),
       list([',', ',', ',', ',']), list([',', ',', ',', ',', ',', ',']),
       list([',', ',', ',', ',', ',', ',', ',']),
       list([',', ',', ',', ',', ',', ',', ',', ',']),
       list([',', ',', ',', ',', ',', ',', ',', ',', ',']),
       list([',', ',', ',', ',', ',', ',', ',', ',', ',', ',']),
       list([',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',']),
       list([',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ',', ','])],
      dtype=object)

Como podemos ver, encontramos alguns elementos com vírgulas. Vamos substituir pelo primeiro diretor, que consideraremos "principal". Caso seja necessária uma análise por diretor, no entanto, talvez seja melhor separar os diretores (o que geraria alguns filmes repetidos, mas isso não seria problema) ou ignorar todos os com vírgula.

In [49]:
df['director'].replace('.*(?<=,)',"",regex=True,inplace=True)

In [50]:
df['country'].replace(',',"",regex=True,inplace=True)

In [51]:
np.unique(df['director'].str.findall(r','))

array([list([])], dtype=object)

Pronto!

 ### Elenco (cast)

Neste caso, naturalmente haverá mais de um ator. Podemos, para uma análise futura, separar cada ator. Agora, a única coisa relevante a se fazer é remover os nulos.

In [53]:
df['cast'].fillna('Unknown',inplace=True)

### Data de Adesão (date_added), Nota (rating) e Duração (duration)

Nestes casos, a quantidade de nulos é tão baixa que nem é relevante considerá-los. Podemos simplesmente removê-los! Como são os últimos que restam, podemos só remover todos os nulos do DataFrame:

In [60]:
df.dropna(inplace=True)

Pronto! Agora podemos verificar quantos nulos temos:

In [61]:
df.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64